# Explainable AI (xAI) Models

For Muscle Group Classification, two models had very high results: SVC and Random Tree Classifier. In order to know the models are not only making the right decisions but also making them for the right reasons, we use xAI models to evaluate it.

In [3]:
import pickle
import dice_ml
from dice_ml.utils import helpers  # Import helper functions
from mediapipe_handler import MediaPipeHandler

In [ ]:
import pandas as pd
import numpy as np

mediapipe_model = MediaPipeHandler()

#training_dataset=mediapipe_model.read_csv_to_pd("/Users/yasinetawfeek/Developer/DesD_AI_pathway/AI/data/train_new.csv")
#testing_dataset=mediapipe_model.read_csv_to_pd("/Users/yasinetawfeek/Developer/DesD_AI_pathway/AI/data/test_new.csv")

training_dataset=mediapipe_model.read_csv_to_pd(r"H:\\DesD_AI_pathway\\AI\\data\\train_new.csv")
testing_dataset=mediapipe_model.read_csv_to_pd(r"H:\\DesD_AI_pathway\\AI\\data\\test_new.csv")

"""
Removes original feature and splits it into x,y,z components

"""
def Preprocess_data(dataframe,columns_to_flatten):
    final_df=dataframe.copy()
    # Expanding each column into 3 separate columns (x, y, z) and appending it to the final dataframe.
    for column in columns_to_flatten:
        # print(np.vstack(dataframe[column]).astype(float))
        expanded_df=pd.DataFrame(np.vstack(dataframe[column]).astype(float), 
                           columns=[column+'_x', column+'_y', column+'_z'],
                           index=dataframe.index)
        new_df = pd.concat([dataframe.drop(column, axis=1), expanded_df], axis=1)
        for new_column in new_df.columns:
            final_df[new_column] = new_df[new_column]

    return final_df.drop(columns=columns_to_flatten,axis=1)

"""
Splits dataset into X_train,y_train or X_test,y_test, if you give it training dataset then X_train and y_train

"""
def Return_X_y(dataframe,columns_to_delete):
    X=dataframe.drop(columns=columns_to_delete)
    y=dataframe['muscle group']
    return X,y

features_to_split=['left_shoulder',
       'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist',
       'right_wrist', 'left_hip', 'right_hip', 'left_knee',
       'right_knee', 'left_ankle', 'right_ankle']

training_dataset_preprocessed=Preprocess_data(training_dataset,features_to_split)
X_train, y_train = Return_X_y(training_dataset_preprocessed,['label','muscle group','image','Unnamed: 0'])


testing_dataset_preprocessed=Preprocess_data(testing_dataset,features_to_split)
X_test, y_test = Return_X_y(testing_dataset_preprocessed,['label','muscle group','image','Unnamed: 0'])

columns_to_drop = ['left_heel_z', 'left_elbow_z', 'right_shoulder_y', 'right_elbow_x', 'right_pinky_z', 'left_ankle_x', 'right_hip_z', 'right_ankle_z', 'right_wrist_z', 'right_pinky_y', 'left_pinky_x', 'left_wrist_x', 'left_foot_index_z', 'right_foot_index_y', 'left_wrist_z', 'right_thumb_x', 'left_index_y', 'left_foot_index_y', 'right_hip_y', 'right_index_z', 'right_foot_index_x', 'left_knee_y', 'right_knee_y', 'right_knee_x', 'left_foot_index_x', 'left_thumb_x', 'right_foot_index_z', 'left_thumb_z', 'right_elbow_z', 'left_heel_x', 'left_ankle_y', 'left_pinky_y', 'left_pinky_z', 'right_thumb_y', 'right_heel_z', 'right_ankle_x', 'right_heel_y', 'right_index_y', 'left_ankle_z', 'left_elbow_x', 'right_index_x', 'right_wrist_y', 'right_elbow_y', 'right_heel_x', 'left_heel_y', 'right_hip_x', 'left_index_z', 'right_wrist_x', 'right_pinky_x', 'left_thumb_y', 'right_thumb_z', 'right_ankle_y', 'left_index_x']

X_train_feature_eng=X_train.drop(columns=columns_to_drop)
X_test_feature_eng=X_test.drop(columns=columns_to_drop)

current OS working directory is h:\DesD_AI_pathway\AI\app


In [ ]:
joints = features_to_split
features = X_train_feature_eng.columns.tolist()

In [ ]:
X_test_feature_eng

In [ ]:
X_test

In [ ]:
with open('rcf_feature_eng_model.pkl', 'rb') as f:
    random_tree_model = pickle.load(f)

# with open('model.pkl', 'rb') as f:
#     svc_model = pickle.load(f)

In [ ]:
import shap
explainer = shap.Explainer(random_tree_model.predict, X_train_feature_eng)
shap_values = explainer(X_test_feature_eng[:20])

In [ ]:
shap_values

In [ ]:
shap.plots.waterfall(shap_values[19], max_display=10)

In [ ]:
data_dice = dice_ml.Data(dataframe=pd.concat([X_train_feature_eng, pd.Series(y_train, name="label")], axis=1),
                         continuous_features=features, outcome_name="label")

model_dice = dice_ml.Model(model=random_tree_model, backend="sklearn")
exp = dice_ml.Dice(data_dice, model_dice, method="random")


In [ ]:
y_test[0]

In [ ]:
query_instance = X_test_feature_eng.iloc[[0]]  # Note the double brackets to keep it as DataFrame

cf = exp.generate_counterfactuals(query_instance, total_CFs=1, desired_class=3)
cf.visualize_as_dataframe()

In [ ]:
X_test_feature_eng.iloc[[0]]

In [ ]:
original = query_instance
counterfactual = cf.cf_examples_list[0].final_cfs_df.iloc[0]

diffs = (counterfactual - original).abs()
important_joints = set()
for joint in joints:
    if any(diffs[f'{joint}_{axis}'] > 0.01 for axis in ['x', 'y', 'z']):
        important_joints.add(joint)

In [ ]:
import matplotlib.pyplot as plt

def plot_skeleton(joint_coords, highlighted_joints=set()):
    connections = [
        ('shoulder', 'elbow'), ('elbow', 'wrist'),
        ('hip', 'knee'), ('shoulder', 'hip')
    ]
    for j1, j2 in connections:
        x = [joint_coords[f'{j1}_x'], joint_coords[f'{j2}_x']]
        y = [joint_coords[f'{j1}_y'], joint_coords[f'{j2}_y']]
        plt.plot(x, y, 'k-', linewidth=2)

    for joint in joints:
        x = joint_coords[f'{joint}_x']
        y = joint_coords[f'{joint}_y']
        color = 'red' if joint in highlighted_joints else 'blue'
        plt.scatter(x, y, c=color, s=100)
        plt.text(x+0.01, y+0.01, joint, fontsize=9)

    plt.gca().invert_yaxis()
    plt.axis('equal')
    plt.title("Skeleton with Important Joints Highlighted")
    plt.show()

plot_skeleton(original.to_dict(), highlighted_joints=important_joints)